In [1]:
import glob, os
import pandas as pd
import pdb
from IPython.core import display as ICD 
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)


ModuleNotFoundError: No module named 'pandas'

Collect files with bad volumes from our text input file:

In [ ]:
# CODE TIP -extend functionally by adding automatic reference to volume check file 

filename='bad_volumes.txt'
vol_dict = {}
with open(filename) as f:
    content=f.readlines()
    for line in content:
        if "training" in line:
            file=line.split("\t")[0]
            vol=line.split("\t")[1].strip()
        else:
            file=line.split("\t\t")[0]
            vol=line.split("\t\t")[1].strip()
        temp_tuple=(file,vol)
        sub_id = file.split(":")[1].split("_")[0]
        if sub_id not in vol_dict:
            vol_dict[sub_id] = [temp_tuple]
        else:
            vol_dict[sub_id].append(temp_tuple)



In [ ]:
## setup scan notes dataframe

df_orig = pd.read_csv("w1_notes.csv", encoding='latin 1')
w1_scan_notes = df_orig[['participantID', 'w1scan_scannotes']]
w1_scan_notes=w1_scan_notes.drop(0)
w1_scan_notes.set_index('participantID', inplace=True)
w1_scan_notes.head()

### Below we list out the files with bad volumes and view the scan notes. 
- View [`CHANGELOG.md`]() for further details on scan if available.

*Note we need to add scan notes file input, currently only lists bad volume files*

In [ ]:
print("Here we look at files found not matching their expected volume. \
      \nExpectations: { rl: 212, training: 233, resting: 147 } \n")
  
print("Subjects with bad files: \t{}".format(sorted(vol_dict.keys())))
for sub in vol_dict:
    # get information from resources to print specific subject information
    bbx_id= sub.replace("sub-", "bbx_")

    train=[x for x in vol_dict[sub] if "training" in x[0]]
    rl=[x for x in vol_dict[sub] if "rl" in x[0]]
    rest=[x for x in vol_dict[sub] if "resting" in x[0]]
    print("\n________________________________________________________________")
    print("\nID: {}\n".format(sub))
    print("> FILES WITH BAD VOLUMES")
    if train:
        print("\n- Training files: \n")
        for file, vol in train:
            ses=file.split("_")[1]
            print("{} \t\t{}".format(file,vol))
    if rl:
        print("\n- Prediction Error files: \n")
        for file, vol in rl:
            print("{} \t\t{}".format(file,vol))
    if rest:
        print("\n- Resting files: \n")
        for file, vol in rest:
            print("{} \t\t{}".format(file,vol))
            
    print("\n> SCAN NOTES")
    ICD.display(w1_scan_notes.loc[bbx_id])
    print("\n\n")


### BIDS Directory Check

In [ ]:
sessions=['ses-1']
for sess_id in sessions:
    if "1" in sess_id:
        print("\n> SESSION 1\n")
    else:
        print("\n> SESSION 2\n")
        
    #ses_mia=pd.read_csv('bro_{}_missing.csv'.format(sess_id),sep="\t")
    #ses_mia.set_index("Unnamed: 0", inplace=True)
    #ses_mia.index.names=["subjects"]
    
    ses_pmia=pd.read_csv('qa/bbx_{}_partial_missing.csv'.format(sess_id),sep="\t")
    ses_pmia.set_index("Unnamed: 0", inplace=True)
    ses_pmia.index.names=["subjects"]
    
    
        
    
    #ses_fd=pd.read_csv('bro_{}_found.csv'.format(sess_id),sep="\t")
    #ses_fd.set_index("Unnamed: 0", inplace=True)
    #ses_fd.index.names=["subjects"]
    
    

    #print("All files found missing: \t{}\n".format(' '.join(ses_mia.index.values)))
    print("Some files found missing in subjects: \n")# \t{}\n".format(' '.join(ses_pmia.index.values)))
    #print("All files found: \t\t{}\n".format(' '.join(ses_fd.index.values)))

    #print("\nDataframe (Missing files): ")
    #ICD.display(ses_mia)
    
    for sub in ses_pmia.index.values:
        print("> {} \n".format(sub))
        bro_id = sub.replace("sub-", "bbx_")
        ICD.display(ses_pmia.loc[sub, :])
        ICD.display(w1_scan_notes.loc[bbx_id])
        print("\n")
    print("\nDataframe (Partial files found): ")
    ICD.display(ses_pmia)
    print("\n")
    
    #print("\nDataframe (All files found): ")
    #ICD.display(ses_fd)
    
    